# Enriching dataset by scraping additional info about vacancies

In [33]:
#from datetime import datetime as dt

import numpy as np
print('Numpy version:', np.__version__)
import pandas as pd
print('Pandas version:', pd.__version__)
import matplotlib as mpl
print('Matplotlib version:', mpl.__version__)
import matplotlib.pyplot as plt
import seaborn as sns
print('Seaborn version:', sns.__version__)

from bs4 import BeautifulSoup
import json

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium import webdriver

import sys

from tqdm import tqdm

Numpy version: 1.23.5
Pandas version: 1.5.2
Matplotlib version: 3.6.2
Seaborn version: 0.12.1


In [2]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

handler = logging.StreamHandler(stream=sys.stdout)
handler.setFormatter(logging.Formatter(fmt='[%(asctime)s: %(funcName)s: %(levelname)s] %(message)s'))
logger.addHandler(handler)

#### Loading dataset to enrich

In [3]:
df_filename = 'datasets/vacancies-no-duplicates-2022-11-27.csv'

In [5]:
df = pd.read_csv(df_filename, index_col=0)
df.head()

,vacancy_name,company_name,address,latitude,longitude,salary_from,salary_to,salary_currency,salary_gross,publication_time,last_changed,schedule,req,resp,cond
vacancy_id,,,,,,,,,,,,,,,
72750262,Ведущий аналитик (качество данных\Data quality),Банк ВТБ (ПАО),"Москва, Пресненская набережная, 10с1",55.748914,37.535466,NaN,NaN,NaN,NaN,2022-11-23 07:08:44,2022-11-23 07:08:44,FULL_DAY,Высшее образование. Опыт работы в качестве ана...,"Мониторинг качества данных, организация процес...",Трудоустройство согласно Законодательству. Кон...
72674818,Аналитик данных,Городские информационные системы,NaN,NaN,NaN,120000.0,180000.0,RUR,False,2022-11-27 09:36:56,2022-11-27 10:21:43,FULL_DAY,"...заданий, валидация данных. Особенности OLAP...","...SQL-запросы, хранимые процедуры, вьюшки. Ви...",Удалённая работа. Уютный коллектив. Минимум бю...
72527010,Аналитик базы данных,Вектор,"Красногорск, бульвар Строителей, 4к1",55.814462,37.385412,70000.0,NaN,RUR,False,2022-11-27 08:32:14,2022-11-27 09:21:16,FULL_DAY,Умение обрабатывать большие объемы данных и ви...,...данных и подготовка информации для расчетов...,"График: 5/2 , с 8:00 до 17:00,выходные: суббот..."
70839955,Аналитик данных (Data Office),"VK, ВКонтакте",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-27 07:02:13,2022-11-27 07:02:13,FULL_DAY,Понимаете основные продуктовые и бизнес-метрик...,Плотно взаимодействовать на ежедневной основе ...,NaN
72943931,Аналитик ГЕО данных,ГАУ ИНСТИТУТ ГЕНПЛАНА МОСКВЫ,"Москва, 2-я Брестская улица, 2/14",55.769641,37.593121,NaN,NaN,NaN,NaN,2022-11-25 13:00:40,2022-11-26 19:50:13,FULL_DAY,Хорошее знание инструментов WEB-картографии и ...,Интеграция данных из различных источников в ед...,"Стабильная заработная плата, включающая оклад,..."


#### Setting up browser

In [8]:
chrome_mode = 'headed' #'headless' # for debug purposes we can change this value to any but 'headless' to run Chrome in standard mode
chrome_options = Options()
if chrome_mode == 'headless':
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--headless')
service = Service(executable_path="d:\\Applications\\WebDriver\\chromedriver-107-x32.exe")
browser = webdriver.Chrome(service=service, options=chrome_options)

In [9]:
base_url = 'https://hh.ru/vacancy/{}'

#### Analyzing single vacancy webpage structure

In [10]:
browser.get(base_url.format(df.index[0]))
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [21]:
#<span class="bloko-tag__section bloko-tag__section_text" data-qa="bloko-tag__text">Английский язык</span>
for item in soup.find_all(attrs={'data-qa': 'bloko-tag__text'}):
    print(item.text)

Анализ данных
Аналитическое мышление
Data quality


In [19]:
soup.find_all(attrs={'data-qa': 'vacancy-description'})[0]

<div class="vacancy-branded-user-content" data-qa="vacancy-description" itemprop="description"><strong>Обязанности:</strong> <ul> <li>участие в проектах по внедрению и развитию инструментов и процессов управления данными;</li> <li>разработка нормативной документации в части управления данными (качеством, доступностью, актуальностью, полнотой, стоимостью и т.д.);</li> <li>выстраивание взаимоотношений с внутренними клиентами в рамках функционально – ролевой модели управления данными;</li> <li>мониторинг качества данных, организация процессов управления инцидентами и проблемами качества данных, реализация мероприятий по повышению качества данных;</li> <li>участие в разработке бизнес - глоссария терминов Банка;</li> </ul> <strong>Требования:</strong> <ul> <li>высшее образование;</li> <li>опыт работы в качестве аналитика не менее 3-х лет;</li> <li>з​нания в области управленческого, бухгалтерского учета, МСФО, риск - отчетности;</li> <li>опыт руководства проектами или участия в проектах по в

Page code analysis shows that we can take key skills from tags where `data-qa="bloko-tag__text"` and vacancy description from tags where `data-qa="vacancy-description"` and vacancy experience needed from tags where `data-qa="vacancy-experience"`.  
We can get cleaned text using _.text_ property

In [24]:
df['full_description'] = ''
df['key_skills'] = ''
df['experience_needed'] = ''
df.head()

,vacancy_name,company_name,address,latitude,longitude,salary_from,salary_to,salary_currency,salary_gross,publication_time,last_changed,schedule,req,resp,cond,full_description,key_skills,experience_needed
vacancy_id,,,,,,,,,,,,,,,,,,
72750262,Ведущий аналитик (качество данных\Data quality),Банк ВТБ (ПАО),"Москва, Пресненская набережная, 10с1",55.748914,37.535466,NaN,NaN,NaN,NaN,2022-11-23 07:08:44,2022-11-23 07:08:44,FULL_DAY,Высшее образование. Опыт работы в качестве ана...,"Мониторинг качества данных, организация процес...",Трудоустройство согласно Законодательству. Кон...,,,
72674818,Аналитик данных,Городские информационные системы,NaN,NaN,NaN,120000.0,180000.0,RUR,False,2022-11-27 09:36:56,2022-11-27 10:21:43,FULL_DAY,"...заданий, валидация данных. Особенности OLAP...","...SQL-запросы, хранимые процедуры, вьюшки. Ви...",Удалённая работа. Уютный коллектив. Минимум бю...,,,
72527010,Аналитик базы данных,Вектор,"Красногорск, бульвар Строителей, 4к1",55.814462,37.385412,70000.0,NaN,RUR,False,2022-11-27 08:32:14,2022-11-27 09:21:16,FULL_DAY,Умение обрабатывать большие объемы данных и ви...,...данных и подготовка информации для расчетов...,"График: 5/2 , с 8:00 до 17:00,выходные: суббот...",,,
70839955,Аналитик данных (Data Office),"VK, ВКонтакте",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-27 07:02:13,2022-11-27 07:02:13,FULL_DAY,Понимаете основные продуктовые и бизнес-метрик...,Плотно взаимодействовать на ежедневной основе ...,NaN,,,
72943931,Аналитик ГЕО данных,ГАУ ИНСТИТУТ ГЕНПЛАНА МОСКВЫ,"Москва, 2-я Брестская улица, 2/14",55.769641,37.593121,NaN,NaN,NaN,NaN,2022-11-25 13:00:40,2022-11-26 19:50:13,FULL_DAY,Хорошее знание инструментов WEB-картографии и ...,Интеграция данных из различных источников в ед...,"Стабильная заработная плата, включающая оклад,...",,,


Let's test and fill both new columns for first 3 rows of dataset => SUCCESSFUL  
Filling all rows...

In [25]:
logger.debug('Beginning parsing vacancies')
total_num = len(df.index)
for i, vac_id in enumerate(df.index):
    logger.debug('Parsing vacancy ID {} ({} / {}) parsing'.format(vac_id, i+1, total_num))
    browser.get(base_url.format(vac_id))
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    descr = soup.find_all(attrs={'data-qa': 'vacancy-description'})
    if len(descr) > 0:
        df.loc[vac_id, 'full_description'] = descr[0].text
    descr = soup.find_all(attrs={'data-qa': 'bloko-tag__text'})
    if len(descr) > 0:
        df.loc[vac_id, 'key_skills'] = ';'.join([i.text for i in descr])
    descr = soup.find_all(attrs={'data-qa': 'vacancy-experience'})
    if len(descr) > 0:
        df.loc[vac_id, 'experience_needed'] = descr[0].text
logger.debug('Parsing finished')
df.head()

[2022-11-27 20:06:22,727: <module>: DEBUG] Beginning parsing vacancies
[2022-11-27 20:06:22,739: <module>: DEBUG] Parsing vacancy ID 72750262 (0 / 435) parsing
[2022-11-27 20:06:25,359: <module>: DEBUG] Parsing vacancy ID 72674818 (1 / 435) parsing
[2022-11-27 20:06:27,460: <module>: DEBUG] Parsing vacancy ID 72527010 (2 / 435) parsing
[2022-11-27 20:06:30,323: <module>: DEBUG] Parsing vacancy ID 70839955 (3 / 435) parsing
[2022-11-27 20:06:32,547: <module>: DEBUG] Parsing vacancy ID 72943931 (4 / 435) parsing
[2022-11-27 20:06:34,729: <module>: DEBUG] Parsing vacancy ID 72677600 (5 / 435) parsing
[2022-11-27 20:06:37,305: <module>: DEBUG] Parsing vacancy ID 72785935 (6 / 435) parsing
[2022-11-27 20:06:39,518: <module>: DEBUG] Parsing vacancy ID 72681497 (7 / 435) parsing
[2022-11-27 20:06:41,378: <module>: DEBUG] Parsing vacancy ID 71424469 (8 / 435) parsing
[2022-11-27 20:06:43,674: <module>: DEBUG] Parsing vacancy ID 72432302 (9 / 435) parsing
[2022-11-27 20:06:46,883: <module>: DEB

,vacancy_name,company_name,address,latitude,longitude,salary_from,salary_to,salary_currency,salary_gross,publication_time,last_changed,schedule,req,resp,cond,full_description,key_skills,experience_needed
vacancy_id,,,,,,,,,,,,,,,,,,
72750262,Ведущий аналитик (качество данных\Data quality),Банк ВТБ (ПАО),"Москва, Пресненская набережная, 10с1",55.748914,37.535466,NaN,NaN,NaN,NaN,2022-11-23 07:08:44,2022-11-23 07:08:44,FULL_DAY,Высшее образование. Опыт работы в качестве ана...,"Мониторинг качества данных, организация процес...",Трудоустройство согласно Законодательству. Кон...,Обязанности: участие в проектах по внедрению ...,Анализ данных;Аналитическое мышление;Data quality,3–6 лет
72674818,Аналитик данных,Городские информационные системы,NaN,NaN,NaN,120000.0,180000.0,RUR,False,2022-11-27 09:36:56,2022-11-27 10:21:43,FULL_DAY,"...заданий, валидация данных. Особенности OLAP...","...SQL-запросы, хранимые процедуры, вьюшки. Ви...",Удалённая работа. Уютный коллектив. Минимум бю...,Мы сейчас переносим отчетность из SAP систем н...,Английский язык;Работа в команде;SQL;PostgreSQ...,3–6 лет
72527010,Аналитик базы данных,Вектор,"Красногорск, бульвар Строителей, 4к1",55.814462,37.385412,70000.0,NaN,RUR,False,2022-11-27 08:32:14,2022-11-27 09:21:16,FULL_DAY,Умение обрабатывать большие объемы данных и ви...,...данных и подготовка информации для расчетов...,"График: 5/2 , с 8:00 до 17:00,выходные: суббот...","Мы группа компаний ""Truck Radar"" - лидеры на р...",Базы данных;Работа с базами данных;Прогнозиров...,1–3 года
70839955,Аналитик данных (Data Office),"VK, ВКонтакте",NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-27 07:02:13,2022-11-27 07:02:13,FULL_DAY,Понимаете основные продуктовые и бизнес-метрик...,Плотно взаимодействовать на ежедневной основе ...,NaN,Мы занимаемся аналитикой экосистемных продукто...,SQL;Python;Математическая статистика;BI;Аналитика,1–3 года
72943931,Аналитик ГЕО данных,ГАУ ИНСТИТУТ ГЕНПЛАНА МОСКВЫ,"Москва, 2-я Брестская улица, 2/14",55.769641,37.593121,NaN,NaN,NaN,NaN,2022-11-25 13:00:40,2022-11-26 19:50:13,FULL_DAY,Хорошее знание инструментов WEB-картографии и ...,Интеграция данных из различных источников в ед...,"Стабильная заработная плата, включающая оклад,...",Обязанности: • Создание и развитие корпоративн...,QGIS;ArcGIS;ГИС;SQL;PostgreSQL,1–3 года


Saving intermediate dataset variant

In [26]:
df.to_csv('datasets/vacancies-no-duplicates-augm-2022-11-27.csv')

In [46]:
browser.quit()

In [27]:
df['experience_needed'].value_counts()

1–3 года        237
3–6 лет         176
более 6 лет      12
не требуется     10
Name: experience_needed, dtype: int64

Determining number of key skills mentioned

In [31]:
df['key_skills_num'] = df['key_skills'].apply(lambda x: len(x.split(';')))
df['key_skills_num'].max()

29

Calculating key skills frequencies

In [65]:
key_skills_freq = {}
for vac_id in tqdm(df.index):
    if df.loc[vac_id, 'key_skills'] != '':
        for skill in df.loc[vac_id, 'key_skills'].split(';'):
            if key_skills_freq.get(skill, None) is None:
                key_skills_freq[skill] = 0
            key_skills_freq[skill] += 1

100%|██████████| 435/435 [00:00<00:00, 22825.07it/s]


In [76]:
df_skills = pd.Series(key_skills_freq)
df_skills

Анализ данных             97
Аналитическое мышление    60
Data quality               2
Английский язык           29
Работа в команде          15
                          ..
Presto                     1
Solr                       1
Elasticsearch              1
Credit Risk                1
Lending                    1
Length: 516, dtype: int64

First 10 most frequently skills

In [77]:
df_skills.sort_values(ascending=False).nlargest(20)

SQL                                    277
Python                                 229
Анализ данных                           97
Аналитическое мышление                  60
MS SQL                                  49
MS PowerPoint                           45
Математическая статистика               43
Machine Learning                        34
Power BI                                34
Hadoop                                  31
Английский язык                         29
Статистический анализ                   27
Работа с большим объемом информации     27
Spark                                   27
MS Excel                                25
PostgreSQL                              24
Big Data                                23
ML                                      23
Data Analysis                           23
Аналитические исследования              23
dtype: int64

In [78]:
df_skills = pd.DataFrame(df_skills, columns=['frequency']).reset_index().rename(columns={'index': 'skill'})
df_skills['skill_class'] = np.nan
df_skills.head()

,skill,frequency,skill_class
0,Анализ данных,97,NaN
1,Аналитическое мышление,60,NaN
2,Data quality,2,NaN
3,Английский язык,29,NaN
4,Работа в команде,15,NaN


In [82]:
m = df_skills['skill'].str.contains('SQL')
df_skills.loc[m, 'skill_class'] = 'SQL'
df_skills.loc[m, ['skill', 'skill_class']]

,skill,skill_class
5,SQL,SQL
6,PostgreSQL,SQL
30,MS SQL,SQL
39,SQL запросы,SQL
65,PL/SQL,SQL
139,NoSQL,SQL
142,PostgeSQL,SQL
204,Основы SQL,SQL
205,Базовые знания SQL,SQL
219,MS SQL Server,SQL


In [118]:
df_skills.loc[[10, 16, 9, 31, 90], 'skill_class'] = 'SQL'

In [86]:
m = df_skills['skill'].str.contains('Python')
df_skills.loc[m, 'skill_class'] = 'Python'
df_skills.loc[m, ['skill', 'skill_class']]

,skill,skill_class
7,Python,Python
143,Python 3.8+,Python


In [94]:
df_skills.loc[[0, 20, 68, 26, 161, 22], 'skill_class'] = 'Analysis & Statistics'

In [110]:
df_skills.loc[[189, 54, 349, 146, 460, 462, 475, 481], 'skill_class'] = 'Machine Learning'

In [114]:
df_skills.loc[[27, 34, 29, 55, 56, 46, 11], 'skill_class'] = 'BI & Presentations'

In [119]:
df_skills[df_skills['skill_class'].isna() & (df_skills['frequency'] > 1)].nlargest(30, columns=['frequency'])

,skill,frequency,skill_class
1,Аналитическое мышление,60,NaN
89,Hadoop,31,NaN
3,Английский язык,29,NaN
62,Работа с большим объемом информации,27,NaN
208,Spark,27,NaN
93,Big Data,23,NaN
53,Pandas,20,NaN
154,Git,20,NaN
177,Data Science,19,NaN
4,Работа в команде,15,NaN


In [122]:
print('Total number of mentions: ', df_skills['frequency'].sum())
print('Unclassified skills (multiple mentions): ', df_skills[df_skills['skill_class'].isna() & (df_skills['frequency'] > 1)]['frequency'].sum())
print('Unclassified skills (single mention): ', df_skills[df_skills['skill_class'].isna() & (df_skills['frequency'] == 1)]['frequency'].sum())

Total number of mentions:  2342
Unclassified skills (multiple mentions):  907
Unclassified skills (single mention):  307


In [123]:
df_skills.groupby(by='skill_class', dropna=False).agg({'frequency': 'sum'}).sort_values(by='frequency', ascending=False)

,frequency
skill_class,
NaN,1214
SQL,435
Python,230
Analysis & Statistics,229
BI & Presentations,140
Machine Learning,94
